# Load Dataset and re-process

In [1]:
from datasets import load_dataset,load_from_disk

In [2]:
# Load dataset
dataset = load_dataset('vicgalle/alpaca-gpt4')

In [3]:
print(dataset['train'])

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})


# Different models training

In [1]:
DI_model_save_name = 'compare_work/DI_GPTRQA_t5_small_5e'
DI_generation_texts_pth = 'compare_work/DI_GTPRQA_gen_5e.txt'
DI_FT_generation_texts_pth = 'compare_work/DI_FT_GPTRQA_gen_5e.txt'

## DI-t5-small

In [2]:
def preprocess_t5_data(example,index):
    if example['output']:
        answer_text = example['output']
    else:
        answer_text = "No answer found"
    return {
        'index':index,
        'input_text': f"answer: {answer_text}",
        'target_text': f"enquiry: {example['instruction']+' '+example['input']}"  # T5需要任务前缀
    }

processed_t5small_dataset = dataset.map(preprocess_t5_data,with_indices=True)

NameError: name 'dataset' is not defined

In [3]:
processed_t5small_dataset['train']['target_text']

NameError: name 'processed_t5small_dataset' is not defined

In [6]:
train_test_split_t5 = processed_t5small_dataset['train'].train_test_split(test_size=0.2)
train_dataset_t5 = train_test_split_t5['train']
val_dataset_t5 = train_test_split_t5['test']

In [7]:
val_dataset_t5

Dataset({
    features: ['instruction', 'input', 'output', 'text', 'index', 'input_text', 'target_text'],
    num_rows: 10401
})

In [8]:
train_dataset_t5.save_to_disk(dataset_path='compare_work/alpaca-0.2-train')
val_dataset_t5.save_to_disk(dataset_path='compare_work/alpaca-0.2-test')

Saving the dataset (0/1 shards):   0%|          | 0/41601 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10401 [00:00<?, ? examples/s]

In [2]:
from datasets import load_dataset,load_from_disk

train_dataset_t5 = load_from_disk('/home/fenghe/Ans2Seq/compare_work/GPTRQA-train')
val_dataset_t5 = load_from_disk('/home/fenghe/Ans2Seq/compare_work/GPTRQA-test')

In [3]:
val_dataset_t5

Dataset({
    features: ['question', 'answer', 'index', 'input_text', 'target_text'],
    num_rows: 7160
})

In [4]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True)
    print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset_t5 = train_dataset_t5.map(tokenize_t5_function, batched=True)
tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)

In [5]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

t5_model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)

In [6]:
from transformers import Trainer, TrainingArguments

training_args_t5 = TrainingArguments(
    output_dir= DI_model_save_name,
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    report_to="none"
)

trainer_t5 = Trainer(
    model=t5_model,
    args=training_args_t5,
    train_dataset=tokenized_train_dataset_t5,
    eval_dataset=tokenized_val_dataset_t5
)


2024-10-08 12:07:28.536236: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 12:07:28.553109: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 12:07:28.553127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 12:07:28.553145: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 12:07:28.557223: I tensorflow/core/platform/cpu_feature_g

In [7]:
trainer_t5.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input_text, answer, target_text, question, index. If input_text, answer, target_text, question, index are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/fenghe/anaconda3/envs/RL4LM2/lib/python3.9/site-packages/transformers-4.18.0-py3.9.egg/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16704
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5220


Epoch,Training Loss,Validation Loss
1,0.218500,0.197530
2,0.203500,0.192061
3,0.195100,0.189502
4,0.189300,0.188421
5,0.185500,0.188121


Saving model checkpoint to compare_work/DI_GPTRQA_t5_small_5e/checkpoint-500
Configuration saved in compare_work/DI_GPTRQA_t5_small_5e/checkpoint-500/config.json
Model weights saved in compare_work/DI_GPTRQA_t5_small_5e/checkpoint-500/pytorch_model.bin
Saving model checkpoint to compare_work/DI_GPTRQA_t5_small_5e/checkpoint-1000
Configuration saved in compare_work/DI_GPTRQA_t5_small_5e/checkpoint-1000/config.json
Model weights saved in compare_work/DI_GPTRQA_t5_small_5e/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input_text, answer, target_text, question, index. If input_text, answer, target_text, question, index are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7160
  Batch size = 16
Saving model checkpoint to compare_work/DI_GPTRQA_t5_small_5e/checkpoint-1500
C

TrainOutput(global_step=5220, training_loss=0.20859477090652875, metrics={'train_runtime': 1062.6779, 'train_samples_per_second': 78.594, 'train_steps_per_second': 4.912, 'total_flos': 1.130374725894144e+16, 'train_loss': 0.20859477090652875, 'epoch': 5.0})

In [8]:
trainer_t5.save_model(DI_model_save_name) 
t5_tokenizer.save_pretrained(DI_model_save_name)

Saving model checkpoint to compare_work/DI_GPTRQA_t5_small_5e
Configuration saved in compare_work/DI_GPTRQA_t5_small_5e/config.json
Model weights saved in compare_work/DI_GPTRQA_t5_small_5e/pytorch_model.bin
tokenizer config file saved in compare_work/DI_GPTRQA_t5_small_5e/tokenizer_config.json
Special tokens file saved in compare_work/DI_GPTRQA_t5_small_5e/special_tokens_map.json


('compare_work/DI_GPTRQA_t5_small_5e/tokenizer_config.json',
 'compare_work/DI_GPTRQA_t5_small_5e/special_tokens_map.json',
 'compare_work/DI_GPTRQA_t5_small_5e/spiece.model',
 'compare_work/DI_GPTRQA_t5_small_5e/added_tokens.json')

In [9]:
def generate_question(answer):
    t5_model.eval()  # 将模型设置为评估模式
    input_ids = t5_tokenizer.encode("answer: " + answer, return_tensors="pt").to(device)
    outputs = t5_model.generate(input_ids, num_beams=5, early_stopping=True)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [10]:
from tqdm import tqdm

# 获取几个样本
samples = val_dataset_t5  
res = []
# 生成问题并比较
with open(DI_generation_texts_pth, 'a') as file:
    for example in tqdm(samples):
        generated_question = generate_question(example['input_text'].replace("answer: ", ""))
        res.append(generated_question.replace("enquiry: ", ""))
        file.write((generated_question.replace("enquiry: ", "")+'\n'))
        # print(f"Answer: {example['input_text'].replace('answer: ', '')}")
        # print(f"Generated Question: {generated_question}")
        # print(f"Actual Question: {example['target_text']}\n")



100%|██████████| 7160/7160 [05:25<00:00, 22.02it/s]


In [11]:
from datasets import load_metric
from nltk.tokenize import word_tokenize
from bert_score import score
from nltk.translate.meteor_score import meteor_score
import evaluate
import pandas as pd
import numpy as np
import torch

def Calmetic(references:list[list[str]], predictions:list[str]):
    '''
    Input format:

    predictions = [
        "What is the capital of France?",
        "Who wrote the book?",
        "What is the largest planet?"
    ]

    references = [
        ["What is the capital city of France?"],
        ["Who is the author of the book?"],
        ["Which planet is the largest in the solar system?"]
    ]
    '''

    # # 加载 BLEU 评分器
    # bleu_metric = load_metric("bleu")

    # # 计算 BLEU 分数
    predictions_tokenized = [word_tokenize(pred) for pred in predictions]
    references_tokenized = [[word_tokenize(refs[0])] for refs in references]
    # B_S = {}
    # for n in range(1, 5):
    #     bleu_metric.add_batch(predictions=predictions_tokenized, references=references_tokenized)
    #     results = bleu_metric.compute(max_order=n)
    #     B_S[f"BLEU-{n}"] = results
    bleu_metric = evaluate.load("bleu")
    B_S = bleu_metric.compute(predictions=predictions, references=references,tokenizer=word_tokenize)
    for i,n in enumerate(B_S['precisions']):
        print(f"BLEU-{i+1} score: {n:.5f}")
        


    # 加载 ROUGE 评分器
    rouge_metric = load_metric("rouge")
    '''
    ROUGE-1: 衡量生成文本和参考文本之间的 unigram 匹配。
    ROUGE-2: 衡量生成文本和参考文本之间的 bigram 匹配。
    ROUGE-L: 衡量生成文本和参考文本之间的最长公共子序列(LCS)。
    ROUGE-Lsum: 基于 LCS 的一个变体，专门用于长文本的评估。
    '''
    # 计算 ROUGE 分数
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)
    rouge1_mid_f1 = rouge_results['rouge1'][1][2]
    rouge2_mid_f1 = rouge_results['rouge2'][1][2]
    rougeL_mid_f1 = rouge_results['rougeL'][1][2]
    rougeLsum_mid_f1 = rouge_results['rougeLsum'][1][2]
    print(f"ROUGE-1 F1 score: {rouge1_mid_f1:.5f}")
    print(f"ROUGE-2 F1 score: {rouge2_mid_f1:.5f}")
    print(f"ROUGE-L F1 score: {rougeL_mid_f1:.5f}")
    print(f"ROUGE-Lsum F1 score: {rougeLsum_mid_f1:.5f}")

    # 计算 METEOR 分数
    meteor_scores = [meteor_score(references=refs, hypothesis=pred) for pred, refs in zip(predictions_tokenized, references_tokenized)]
    average_meteor_score = sum(meteor_scores) / len(meteor_scores)
    print(f"Average METEOR score: {average_meteor_score:.5f}")

    # 计算 BERTScore 分数
    '''
    同样效果：
    bert_metric = load_metric("bertscore",cache_dir="/media/fenghe/New Volume/A2Q/Metric")
    bert_results = bert_metric.compute(predictions=predictions, references=references,lang="en",device=f"cuda:{torch.cuda.device_count() - 1}")

    设置 verbose=True 会使函数在计算过程中输出更多的信息，例如处理进度、当前正在处理的数据等。
    '''
    P, R, F1 = score(predictions, [ref[0] for ref in references], lang="en", verbose=False)
    average_bert_score = F1.mean().item()
    print(f"Average BERTScore F1: {average_bert_score:.5f}")

    return {
        "BLEU":B_S,
        "ROUGE":rouge_results,
        "METERO":meteor_scores,
        "BERTScore":{"Precision":P,"Recall":R,"F1":F1},
    }

In [12]:
with open(DI_generation_texts_pth, 'r') as file:
    content = file.readlines()

In [13]:
refs = [ [i.replace('enquiry: ',"")] for i in val_dataset_t5['target_text']]

In [14]:
refs

[["Why do electronics stop working after they are submerged by water ? I could understand if they have power running through them , but what about electronics that are powered off completely ? Explain like I'm five."],
 ['what are k cups'],
 ['Self Assessment UK - Goods and services for your own use'],
 ["Why do police officers have bullet proof vests for covering vital organs when often times they leave their heads unprotected ? I understand that SWAT teams and riot folks have helmets , but what about regular police ? Explain like I'm five."],
 ["How does an anarcho - socialist society prevent a resurgence of capitalism , and how does an anarchist society in general prevent crime ? To me anarchy seems like a good idea in theory but has fundamental problems with implementation . Please explain like I'm five."],
 ["[ META ] Why are people suddenly usingto ask loaded questions and make political statements ? Then cutely try to make it sound like a genuine question by saying something lik

In [15]:
res = Calmetic(references=refs,predictions=content)

BLEU-1 score: 0.56694
BLEU-2 score: 0.29616
BLEU-3 score: 0.19084
BLEU-4 score: 0.13297


/tmp/ipykernel_79035/924731710.py:46: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/home/fenghe/anaconda3/envs/RL4LM2/lib/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ROUGE-1 F1 score: 0.36712
ROUGE-2 F1 score: 0.21157
ROUGE-L F1 score: 0.33487
ROUGE-Lsum F1 score: 0.33484
Average METEOR score: 0.26859


loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /home/fenghe/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file https://huggingface.co/roberta-larg

Average BERTScore F1: 0.88857


In [16]:
print(res['BLEU'])
print(res['ROUGE']['rougeLsum'][1])

{'bleu': 0.03853358176244415, 'precisions': [0.5669351162437319, 0.29616164708583675, 0.1908358368194963, 0.13297233189244861], 'brevity_penalty': 0.15082333196776876, 'length_ratio': 0.34582378359587534, 'translation_length': 85553, 'reference_length': 247389}
Score(precision=0.5435409432554532, recall=0.2749334379201049, fmeasure=0.3348429587283891)


In [17]:
val_dataset_t5['target_text']

["enquiry: Why do electronics stop working after they are submerged by water ? I could understand if they have power running through them , but what about electronics that are powered off completely ? Explain like I'm five.",
 'enquiry: what are k cups',
 'enquiry: Self Assessment UK - Goods and services for your own use',
 "enquiry: Why do police officers have bullet proof vests for covering vital organs when often times they leave their heads unprotected ? I understand that SWAT teams and riot folks have helmets , but what about regular police ? Explain like I'm five.",
 "enquiry: How does an anarcho - socialist society prevent a resurgence of capitalism , and how does an anarchist society in general prevent crime ? To me anarchy seems like a good idea in theory but has fundamental problems with implementation . Please explain like I'm five.",
 "enquiry: [ META ] Why are people suddenly usingto ask loaded questions and make political statements ? Then cutely try to make it sound like

In [18]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity as torch_cosine_similarity
from sentence_transformers import SentenceTransformer, util

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')#SentenceTransformer("bert-base-uncased") 

reference_texts_ = [ i.replace('enquiry: ',"") for i in val_dataset_t5['target_text'] ]
embeddings1 = sentence_model.encode(content, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_2 = util.pytorch_cos_sim(embeddings1, embeddings2)   #[52002,52002]维度的矩阵，对角线上的值为对应文本的余弦相似度

# 输出余弦相似度的值
print(f"Average Cosine Similarity: {cosine_scores_2.diagonal().mean()}")
print(f"Biggest Cosine Similarity: {cosine_scores_2.diagonal().max()}")
print(f"Middle Cosine Similarity: {cosine_scores_2.diagonal().median()}")


loading configuration file /home/fenghe/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/home/fenghe/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/fenghe/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint wei

Average Cosine Similarity: 0.7585591673851013
Biggest Cosine Similarity: 1.0000008344650269
Middle Cosine Similarity: 0.780834972858429


## -bart-large

In [ ]:

def preprocess_squad(example):
    # Invert the dataset by treating the answer as input and the question as output
    if example['answers']['text']:
    # SQuAD has answers as a list of possible answer texts; we'll just use the first one for simplicity
        answer_text = example['answers']['text'][0]
    else:
        answer_text = "No answer found"
    return {
        'input_text': f"answer: {answer_text}",
        'target_text': example['question']
    }


# Preprocess the dataset
processed_dataset = dataset.map(preprocess_squad)

In [ ]:
train_test_split = processed_dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [ ]:
print(train_dataset.shape)
print(val_dataset.shape)

In [ ]:
from transformers import BartTokenizer

bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

def tokenize_bart_function(examples):
    model_inputs = bart_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = bart_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset_bart = train_dataset.map(tokenize_bart_function, batched=True)
tokenized_val_dataset_bart = val_dataset.map(tokenize_bart_function, batched=True)

In [10]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from transformers import BartForConditionalGeneration

bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to(device)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args_bart = TrainingArguments(
    output_dir='./results_bart_2',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    report_to="none"
)

trainer_bart = Trainer(
    model=bart_model,
    args=training_args_bart,
    train_dataset=tokenized_train_dataset_bart,
    eval_dataset=tokenized_val_dataset_bart
)


In [ ]:
trainer_bart.train()

In [47]:
def generate_question(answer):
    bart_model.eval()  # 将模型设置为评估模式
    input_ids = bart_tokenizer.encode("answer: " + answer, return_tensors="pt").to(device)
    outputs = bart_model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    question = bart_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [ ]:
# 获取几个样本
samples = val_dataset.shuffle(seed=42).select(range(5))  # 随机选择5个样本

# 生成问题并比较
for example in samples:
    generated_question = generate_question(example['input_text'].replace("answer: ", ""))
    print(f"Answer: {example['input_text'].replace('answer: ', '')}")
    print(f"Generated Question: {generated_question}")
    print(f"Actual Question: {example['target_text']}\n")


## DI+FT

In [16]:
from datasets import load_from_disk

#train_dataset_t5 = load_from_disk('/home/fenghe/Ans2Seq/compare_work/alpaca-0.2-train')
val_dataset_t5 = load_from_disk('/home/fenghe/Ans2Seq/compare_work/GPTRQA-test')

In [17]:
import pandas as pd
pd.DataFrame(val_dataset_t5)

,question,answer,index,input_text,target_text
0,Why do electronics stop working after they are...,"Water is a conductor of electricity, which mea...",0,"answer: Water is a conductor of electricity, w...",enquiry: Why do electronics stop working after...
1,what are k cups,K-Cups are single-serve coffee pods that are u...,1,answer: K-Cups are single-serve coffee pods th...,enquiry: what are k cups
2,Self Assessment UK - Goods and services for yo...,"In the UK, self assessment is a system used by...",2,"answer: In the UK, self assessment is a system...",enquiry: Self Assessment UK - Goods and servic...
3,Why do police officers have bullet proof vests...,Police officers wear bulletproof vests to prot...,3,answer: Police officers wear bulletproof vests...,enquiry: Why do police officers have bullet pr...
4,How does an anarcho - socialist society preven...,An anarcho-socialist society is one in which t...,4,answer: An anarcho-socialist society is one in...,enquiry: How does an anarcho - socialist socie...
...,...,...,...,...,...
7155,Can someone explain American football to me li...,Sure! American football is a sport played by t...,7155,answer: Sure! American football is a sport pla...,enquiry: Can someone explain American football...
7156,Can I get a mortgage from a foreign bank?,chat.openai.comChecking if the site connection...,7156,answer: chat.openai.comChecking if the site co...,enquiry: Can I get a mortgage from a foreign b...
7157,What happens when a (Internation) Central secu...,A central securities depository (CSD) is an in...,7157,answer: A central securities depository (CSD) ...,enquiry: What happens when a (Internation) Cen...
7158,"Does our Moon , Sun or Solar System have an ac...",Our Moon is simply called the Moon. It does no...,7158,answer: Our Moon is simply called the Moon. It...,"enquiry: Does our Moon , Sun or Solar System h..."


In [18]:
from transformers import T5Tokenizer

t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize_t5_function(examples):
    model_inputs = t5_tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=512)
    labels = t5_tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

#tokenized_train_dataset_t5 = train_dataset_t5.map(tokenize_t5_function, batched=True)
tokenized_val_dataset_t5 = val_dataset_t5.map(tokenize_t5_function, batched=True)


In [19]:
import torch
from transformers import T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DI_FT_t5_base_model = T5ForConditionalGeneration.from_pretrained('/home/fenghe/RL4LMs/DI_FT_GPTQRA_diffalg/a2c_50ft_rouge/model').to(device)


In [20]:
def generate_question(answer):
    DI_FT_t5_base_model.eval()  # 将模型设置为评估模式
    input_ids = t5_tokenizer.encode("answer: " + answer, return_tensors="pt").to(device)
    outputs = DI_FT_t5_base_model.generate(input_ids, num_beams=5, early_stopping=True)
    question = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return question

In [21]:
# 获取几个样本
samples = val_dataset_t5.shuffle(seed=42).select(range(5))  # 随机选择5个样本

# 生成问题并比较
for example in samples:
    generated_question = generate_question(example['input_text'].replace("answer: ", ""))
    print(f"Answer: {example['input_text'].replace('answer: ', '')}")
    print(f"Generated Question: {generated_question}")
    print(f"Actual Question: {example['target_text']}\n")


Answer: There can definitely be differences in quality between different brands of corn flakes, even though they may seem like a simple product. The main difference is usually the ingredients that are used. Real Kellogg's corn flakes are made with high-quality ingredients, such as corn, sugar, and salt. The cheaper brands may use lower-quality ingredients or substitute some ingredients with cheaper alternatives. For example, they may use a lower-grade corn or add more sugar to make the cereal taste better. Another difference is in the manufacturing process. Kellogg's has a reputation for using strict quality control measures to ensure that their products meet high standards. They may also use more expensive equipment and techniques to produce their cereals. On the other hand, cheaper brands may cut corners in the manufacturing process to save money, which can lead to lower quality products. In general, you get what you pay for when it comes to products like corn flakes. If you're willi

In [22]:
from tqdm import tqdm

# 获取几个样本
samples = val_dataset_t5  # 随机选择5个样本
res = []
# 生成问题并比较
with open('/home/fenghe/Ans2Seq/compare_work/DI_FT_rouge_GTPRQA_gen_0_2_dffalg_a2c_30e.txt', 'a') as file:
    for example in tqdm(samples):
        generated_question = generate_question(example['input_text'].replace("answer: ", ""))
        res.append(generated_question.replace("enquiry: ", ""))
        file.write((generated_question.replace("enquiry: ", "")+'\n'))
        # print(f"Answer: {example['input_text'].replace('answer: ', '')}")
        # print(f"Generated Question: {generated_question}")
        # print(f"Actual Question: {example['target_text']}\n")



100%|██████████| 7160/7160 [05:28<00:00, 21.77it/s]


In [23]:
from datasets import load_metric
from nltk.tokenize import word_tokenize
from bert_score import score
from nltk.translate.meteor_score import meteor_score
import evaluate
import pandas as pd
import numpy as np
import torch

def Calmetic(references:list[list[str]], predictions:list[str]):
    '''
    Input format:

    predictions = [
        "What is the capital of France?",
        "Who wrote the book?",
        "What is the largest planet?"
    ]

    references = [
        ["What is the capital city of France?"],
        ["Who is the author of the book?"],
        ["Which planet is the largest in the solar system?"]
    ]
    '''

    # # 加载 BLEU 评分器
    # bleu_metric = load_metric("bleu")

    # # 计算 BLEU 分数
    predictions_tokenized = [word_tokenize(pred) for pred in predictions]
    references_tokenized = [[word_tokenize(refs[0])] for refs in references]
    # B_S = {}
    # for n in range(1, 5):
    #     bleu_metric.add_batch(predictions=predictions_tokenized, references=references_tokenized)
    #     results = bleu_metric.compute(max_order=n)
    #     B_S[f"BLEU-{n}"] = results
    bleu_metric = evaluate.load("bleu")
    B_S = bleu_metric.compute(predictions=predictions, references=references,tokenizer=word_tokenize)
    for i,n in enumerate(B_S['precisions']):
        print(f"BLEU-{i+1} score: {n:.5f}")
        


    # 加载 ROUGE 评分器
    rouge_metric = load_metric("rouge")
    '''
    ROUGE-1: 衡量生成文本和参考文本之间的 unigram 匹配。
    ROUGE-2: 衡量生成文本和参考文本之间的 bigram 匹配。
    ROUGE-L: 衡量生成文本和参考文本之间的最长公共子序列(LCS)。
    ROUGE-Lsum: 基于 LCS 的一个变体，专门用于长文本的评估。
    '''
    # 计算 ROUGE 分数
    rouge_results = rouge_metric.compute(predictions=predictions, references=references)
    rouge1_mid_f1 = rouge_results['rouge1'][1][2]
    rouge2_mid_f1 = rouge_results['rouge2'][1][2]
    rougeL_mid_f1 = rouge_results['rougeL'][1][2]
    rougeLsum_mid_f1 = rouge_results['rougeLsum'][1][2]
    print(f"ROUGE-1 F1 score: {rouge1_mid_f1:.5f}")
    print(f"ROUGE-2 F1 score: {rouge2_mid_f1:.5f}")
    print(f"ROUGE-L F1 score: {rougeL_mid_f1:.5f}")
    print(f"ROUGE-Lsum F1 score: {rougeLsum_mid_f1:.5f}")

    # 计算 METEOR 分数
    meteor_scores = [meteor_score(references=refs, hypothesis=pred) for pred, refs in zip(predictions_tokenized, references_tokenized)]
    average_meteor_score = sum(meteor_scores) / len(meteor_scores)
    print(f"Average METEOR score: {average_meteor_score:.5f}")

    # 计算 BERTScore 分数
    '''
    同样效果：
    bert_metric = load_metric("bertscore",cache_dir="/media/fenghe/New Volume/A2Q/Metric")
    bert_results = bert_metric.compute(predictions=predictions, references=references,lang="en",device=f"cuda:{torch.cuda.device_count() - 1}")

    设置 verbose=True 会使函数在计算过程中输出更多的信息，例如处理进度、当前正在处理的数据等。
    '''
    P, R, F1 = score(predictions, [ref[0] for ref in references], lang="en", verbose=False)
    average_bert_score = F1.mean().item()
    print(f"Average BERTScore F1: {average_bert_score:.5f}")

    return {
        "BLEU":B_S,
        "ROUGE":rouge_results,
        "METERO":meteor_scores,
        "BERTScore":{"Precision":P,"Recall":R,"F1":F1},
    }

In [24]:
with open('/home/fenghe/Ans2Seq/compare_work/DI_FT_rouge_GTPRQA_gen_0_2_dffalg_a2c_30e.txt', 'r') as file:
    content = file.readlines()

In [25]:
refs = [ [i.replace('enquiry: ',"")] for i in val_dataset_t5['target_text']]

In [26]:
refs

[["Why do electronics stop working after they are submerged by water ? I could understand if they have power running through them , but what about electronics that are powered off completely ? Explain like I'm five."],
 ['what are k cups'],
 ['Self Assessment UK - Goods and services for your own use'],
 ["Why do police officers have bullet proof vests for covering vital organs when often times they leave their heads unprotected ? I understand that SWAT teams and riot folks have helmets , but what about regular police ? Explain like I'm five."],
 ["How does an anarcho - socialist society prevent a resurgence of capitalism , and how does an anarchist society in general prevent crime ? To me anarchy seems like a good idea in theory but has fundamental problems with implementation . Please explain like I'm five."],
 ["[ META ] Why are people suddenly usingto ask loaded questions and make political statements ? Then cutely try to make it sound like a genuine question by saying something lik

In [27]:
res = Calmetic(references=refs,predictions=content)

BLEU-1 score: 0.58823
BLEU-2 score: 0.31438
BLEU-3 score: 0.20351
BLEU-4 score: 0.14185


/home/fenghe/anaconda3/envs/RL4LM2/lib/python3.9/site-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ROUGE-1 F1 score: 0.37945
ROUGE-2 F1 score: 0.22145
ROUGE-L F1 score: 0.34584
ROUGE-Lsum F1 score: 0.34598
Average METEOR score: 0.27847


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average BERTScore F1: 0.89097


In [28]:
print(res['BLEU']['precisions'])
print(res['ROUGE']['rougeL'][1])

[0.5882318252582913, 0.31438398515999383, 0.20350792547077437, 0.1418530452362901]
Score(precision=0.5623428284834074, recall=0.2820978083732174, fmeasure=0.34584129112556244)


In [29]:
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity as torch_cosine_similarity
from sentence_transformers import SentenceTransformer, util

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')#SentenceTransformer("bert-base-uncased") 

reference_texts_ = [ i.replace('enquiry: ',"") for i in val_dataset_t5['target_text'] ]
embeddings1 = sentence_model.encode(content, convert_to_tensor=True)
embeddings2 = sentence_model.encode(reference_texts_, convert_to_tensor=True)

cosine_scores_2 = util.pytorch_cos_sim(embeddings1, embeddings2)   #[52002,52002]维度的矩阵，对角线上的值为对应文本的余弦相似度

# 输出余弦相似度的值
print(f"Average Cosine Similarity: {cosine_scores_2.diagonal().mean()}")
print(f"Biggest Cosine Similarity: {cosine_scores_2.diagonal().max()}")
print(f"Middle Cosine Similarity: {cosine_scores_2.diagonal().median()}")


Average Cosine Similarity: 0.7703564763069153
Biggest Cosine Similarity: 1.0000008344650269
Middle Cosine Similarity: 0.7930443286895752
